In [1]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import os
import json


# Importar modelos
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

<center><img src=
"ml_lifecycle.jpeg"
width="70%"></center>


In [2]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import os
import json


# Importar modelos
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

---
# 1. Data collection, Data Cleaning & Feature Engineering

### Método de **Feature Engeneering** e **Limpeza de Dados**

In [3]:
def feature_engeneering(df):
    df_eng = df.copy()

    # --- LIMPEZA INICIAL ---
    df_eng = df_eng.drop_duplicates()
    df_eng['hp'] = df['engine'].str.extract(r'(\d+\.?\d*)HP', expand=False).astype(float)
    df_eng['liters'] = df['engine'].str.extract(r'(\d+\.?\d*)L\s', expand=False).astype(float)

    # --- Idade e Uso ---
    var_ano_atual = date.today().year
    df_eng['car_age'] = var_ano_atual - df_eng['model_year']
    df_eng['car_age'] = df_eng['car_age'].replace(0, 1)

    # --- Cilindrada ---
    df_eng['cylinders'] = df['engine'].str.extract(r'(\d+)\s+Cylinder', expand=False)
    df_eng['cylinders'] = df_eng['cylinders'].fillna(df['engine'].str.extract(r'V(\d+)', expand=False))
    df_eng['cylinders'] = df_eng['cylinders'].astype(float)

    # --- Tecnologias de Motor ---
    df_eng['is_turbo'] = df['engine'].str.contains(r'(?i)turbo', na=False).astype(int)
    df_eng['turbo_type'] = df['engine'].str.extract(r'(Twin Turbo|Turbo)', expand=False)
    df_eng['valve_train'] = df['engine'].str.extract(r'(DOHC|OHV|SOHC)', expand=False) 
    df_eng['fuel_injection'] = df['engine'].str.extract(r'(PDI|GDI|MPFI)', expand=False)

    # Miles per year
    df_eng['miles_p_year'] = df_eng['milage'] / df_eng['car_age']

    # --- FUEL TYPE ---
    def clean_fuel(val):
        s = str(val).lower()
        if 'hybrid' in s:
            return 'Hybrid'
        elif 'not supported' in s:
            return 'EV'
        else:
            return val
    df_eng['fuel_type'] = df_eng['fuel_type'].apply(clean_fuel)

    # --- TRANSMISSION TYPE ---
    def clean_transmission(val):
        s = str(val).lower()
        if 'automatic' in s or 'a/t' in s or 'cvt' in s:
            return 'Automatico'
        elif 'manual' in s or 'm/t' in s:
            return 'Manual'
        else:
            return 'Outro'
    df_eng['transmission_type'] = df_eng['transmission'].apply(clean_transmission)

    # --- Cores ---
    top_ext_colors = df_eng['ext_col'].value_counts().nlargest(10).index
    def simplificar_cor_ext(cor):
        return cor if cor in top_ext_colors else 'Other'
    df_eng['ext_col_simple'] = df_eng['ext_col'].apply(simplificar_cor_ext)

    top_int_colors = df_eng['int_col'].value_counts().nlargest(10).index
    def simplificar_cor_int(cor):
        return cor if cor in top_int_colors else 'Other'
    df_eng['int_col_simple'] = df_eng['int_col'].apply(simplificar_cor_int)

    # --- Tratamento de Nulos ---
    cols_texto = df_eng.select_dtypes(include=['object']).columns
    df_eng[cols_texto] = df_eng[cols_texto].replace('-', 'Unknown').fillna('Unknown')
    df_eng['clean_title'] = df_eng['clean_title'].replace('Unknown', 'No')

    # --- Acidente ---
    def verificar_acidente(valor):
        return 0 if 'None' in str(valor) else 1
    df_eng['accident_clean'] = df_eng['accident'].apply(verificar_acidente)

    # 1. Eficiência do motor
    # Evitar divisão por zero somando um valor ínfimo
    df_eng['hp_per_liter'] = df_eng['hp'] / (df_eng['liters'] + 0.001)

    # 2. Rácio de Potência por Cilindro
    df_eng['hp_per_cylinder'] = df_eng['hp'] / (df_eng['cylinders'] + 0.001)

    # 3. Quilometragem (Milage)
    # A milage tem uma distribuição muito "cauda longa". O Log ajuda o modelo a ver melhor as diferenças.
    df_eng['milage_log'] = np.log1p(df_eng['milage'])

    return df_eng

In [4]:
# Recolha os Dados
df_treino = pd.read_csv('dados/train.csv', index_col='id')
df_teste = pd.read_csv('dados/test.csv', index_col='id')

print("✓ Dados lidos com Sucesso!")

✓ Dados lidos com Sucesso!


In [5]:
# Aplicar feature engineering e Limpeza de dados
df_treino_eng = feature_engeneering(df_treino)
df_teste_eng = feature_engeneering(df_teste)

print("✓ Feature Engineering Aplicado com Sucesso!")

✓ Feature Engineering Aplicado com Sucesso!


In [6]:
# Separar target
y = np.log1p(df_treino_eng['price'])
X = df_treino_eng
X_test = df_teste_eng.copy()

# Tipos de Features Relevantes para Previsao
features_numericas = ['hp', 'liters', 'car_age', 'cylinders', 'miles_p_year','milage', 'model_year', 'is_turbo']

features_categoricas = ['brand', 'model', 'fuel_type', 'transmission_type', 
                           'ext_col_simple', 'int_col_simple', 'clean_title', 
                           'turbo_type', 'valve_train', 'fuel_injection']

# Criar dataset numérico
X_num = X[features_numericas].fillna(0)
X_test_num = X_test[features_numericas].fillna(0)

# Encondificar features categoricas | (Ordinal Enconding) Transform each category into a number
X_cat = X[features_categoricas].copy()
X_test_cat = X_test[features_categoricas].copy()

for col in features_categoricas:
    le = LabelEncoder()
    # Fit no treino
    X_cat[col] = X_cat[col].astype(str)
    le.fit(X_cat[col])
    X_cat[col] = le.transform(X_cat[col])

    # Transform no teste (tratando categorias novas)
    X_test_cat[col] = X_test_cat[col].astype(str)
    X_test_cat[col] = X_test_cat[col].apply(
        lambda x: le.transform([x])[0] if x in le.classes_ else -1
    )

# Concatenar features
X_final = pd.concat([X_num, X_cat], axis=1)
X_test_final = pd.concat([X_test_num, X_test_cat], axis=1)

In [7]:
X_final.head(5)

,hp,liters,car_age,cylinders,miles_p_year,milage,model_year,is_turbo,brand,model,fuel_type,transmission_type,ext_col_simple,int_col_simple,clean_title,turbo_type,valve_train,fuel_injection
id,,,,,,,,,,,,,,,,,,
0,172.0,1.6,19,4.0,11210.526316,213000,2007,0,31,495,3,0,7,4,1,2,3,3
1,252.0,3.9,24,8.0,5968.750000,143250,2002,0,28,930,3,0,9,0,1,2,3,3
2,320.0,5.3,24,8.0,5697.125000,136731,2002,0,9,1575,1,0,1,4,1,2,3,3
3,420.0,5.0,9,8.0,2166.666667,19500,2017,0,16,758,3,2,0,1,1,2,3,3
4,208.0,2.0,5,4.0,1477.600000,7388,2021,0,36,1077,3,0,0,0,1,2,3,3


In [8]:
X_test_final.head(5)

,hp,liters,car_age,cylinders,miles_p_year,milage,model_year,is_turbo,brand,model,fuel_type,transmission_type,ext_col_simple,int_col_simple,clean_title,turbo_type,valve_train,fuel_injection
id,,,,,,,,,,,,,,,,,,
188533,240.0,2.0,11,4.0,8909.090909,98000,2015,0,26,1390,3,0,10,0,1,2,3,3
188534,395.0,3.0,6,6.0,1523.666667,9142,2020,0,26,1377,4,0,9,1,1,2,3,3
188535,0.0,3.5,4,6.0,7030.250000,28121,2022,1,14,636,3,0,10,3,0,1,0,2
188536,0.0,0.0,10,0.0,6125.800000,61258,2016,0,3,182,3,0,7,1,0,2,3,3
188537,252.0,2.0,8,4.0,7375.000000,59000,2018,0,3,181,3,0,4,1,1,2,3,3


---
# 2. Model Training & Evaluation

### Método de **Definição dos Modelos** - Dicionário com objetos dos modelos de previsão.

In [9]:
def obter_modelos():
    modelos = {
        #'Linear Regression': LinearRegression(),

        'KNN': KNeighborsRegressor(),

        #'Decision Tree': DecisionTreeRegressor(random_state=42),

        #'Random Forest': RandomForestRegressor(random_state=42, n_jobs=-1),

        #'XGBoost': xgb.XGBRegressor(random_state=42, n_jobs=-1),

        #'SVR Linear': SVR(kernel='linear'),

        #'SVR RBF': SVR(kernel='rbf'),

        #'MLP Small': MLPRegressor(random_state=42, early_stopping=True),

        #'MLP Deep': MLPRegressor(random_state=42, early_stopping=True)
    }

    return modelos

### Métodos para **"GridSearchCV"** - Otimização de Hipérparametrso

In [10]:
def obter_params_grid(nome_modelo):
    grids = {
        'Random Forest': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        'XGBoost': {
            'n_estimators': [100, 500, 1000],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7],
            'subsample': [0.7, 0.9, 1.0],
            'colsample_bytree': [0.7, 0.9, 1.0]
        },
        'KNN': {
            'n_neighbors': [3, 5, 7, 11],
            'weights': ['uniform', 'distance'],
            'p': [1, 2] # 1=Manhattan, 2=Euclidean
        },
        'MLP Deep': {
            'hidden_layer_sizes': [(50,50,50), (100,50), (100,)],
            'activation': ['tanh', 'relu'],
            'alpha': [0.0001, 0.05],
        },
        'SVR RBF': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.1, 0.01],
            'epsilon': [0.1, 0.2, 0.5]
        }
    }
    return grids.get(nome_modelo, {})

def executar_grid_search(modelo, nome_modelo, X, y, usar_scaled=False):
    print(f"\nA iniciar GridSearch no modelo:'{nome_modelo}' ")

    # 1. Obter o grid de parâmetros
    param_grid = obter_params_grid(nome_modelo)

    if not param_grid:
        print(f"‼ ATENÇÃO - Nenhum grid de hiperparametros definido para {nome_modelo} !!")
        return modelo

    # 2. Configurar o GridSearch
    # cv -> número de folds no Cross Validation ("Mais folds = Mais tempo de execução")
    grid_search = GridSearchCV(
        estimator=modelo,
        param_grid=param_grid,
        cv=3, 
        scoring='neg_root_mean_squared_error',
        verbose=1,
        n_jobs=-1  # Usa todos os processadores
    )

    # 3. Treinar
    grid_search.fit(X, y)

    # 4. Resultados
    print(f"✓ Melhores Parâmetros: {grid_search.best_params_}")
    print(f"  Melhor RMSE (CV): {-grid_search.best_score_:,.2f}")
    return grid_search.best_estimator_

### Método de **Avaliação de Modelo**

In [11]:
def avaliar_modelo(modelo, X_train, y_train, X_val, y_val, nome_modelo):

    
    print(f"\n{'='*60}")
    print(f"A Treinar o modelo: '{nome_modelo}'")
    print(f"{'='*60}")

    # Treinar
    modelo.fit(X_train, y_train)

    # Predições
    y_train_pred = modelo.predict(X_train)
    y_val_pred = modelo.predict(X_val)

    # Métricas de treino
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

    # Métricas de validação
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

    # Exibir resultados
    print(f"\n▶ MÉTRICAS DE TREINO:")
    print(f"  RMSE: {train_rmse:,.2f}")

    print(f"\n▶ MÉTRICAS DE VALIDAÇÃO:")
    print(f"  RMSE: {val_rmse:,.2f}")

    return {
        'modelo': nome_modelo,
        'train_rmse': train_rmse,
        'val_rmse': val_rmse,
        'modelo_treinado': modelo
    }

---

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_final, y, test_size=0.2, random_state=42)

In [13]:
# Normalizar dados (importante para KNN, SVM e MLP) | Permite que todas as features contribuam equilibradamente para o modelo
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Alinhar as colunas do conjunto de teste com as colunas usadas no treino
# (mesma ordem; preencher colunas em falta com 0)
X_test_aligned = X_test_final.reindex(columns=X_train.columns, fill_value=0)

X_test_scaled = scaler.transform(X_test_aligned)


In [14]:
print(f"✓ Dados preparados:")
print(f"  - Treino: {X_train.shape[0]} amostras")
print(f"  - Validação: {X_val.shape[0]} amostras")
print(f"  - Features: {X_train.shape[1]}")

✓ Dados preparados:
  - Treino: 150826 amostras
  - Validação: 37707 amostras
  - Features: 18


In [15]:
modelos = obter_modelos()

#Treinar os modelos e Avaliar
resultados = []
modelos_treinados = {}

for nome, modelo in modelos.items():
    # Decidir se usar dados normalizados
    usar_scaled = nome in ['KNN', 'SVR Linear', 'SVR RBF', 'MLP Small', 'MLP Deep']

    # Executar Grid Search para tuning
    if usar_scaled:
        modelo = executar_grid_search(modelo, nome, X_train_scaled, y_train)
        res = avaliar_modelo(modelo, X_train_scaled, y_train, X_val_scaled, y_val, nome)
    else:
        modelo = executar_grid_search(modelo, nome, X_train, y_train)
        res = avaliar_modelo(modelo, X_train, y_train, X_val, y_val, nome)

    resultados.append(res)
    modelos_treinados[nome] = {
        'modelo': res['modelo_treinado'],
        'usar_scaled': usar_scaled
    }

# Comparação final
print(f"\n\n{'='*80}")
print(" ■ COMPARAÇÃO DE MODELOS (ordenados por RMSE de validação)")
print(f"{'='*80}\n")

df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values('val_rmse', ascending=True)

print(df_resultados[['modelo', 'val_rmse']].to_string(index=False))

# Melhor modelo
melhor_resultado = df_resultados.iloc[0]
print(f"\n\n ▶ MELHOR MODELO: {melhor_resultado['modelo']}")
print(f"  RMSE Validação: {melhor_resultado['val_rmse']:,.2f}")


A iniciar GridSearch no modelo:'KNN' 
Fitting 3 folds for each of 16 candidates, totalling 48 fits
✓ Melhores Parâmetros: {'n_neighbors': 11, 'p': 1, 'weights': 'uniform'}
  Melhor RMSE (CV): 0.52

A Treinar o modelo: 'KNN'

▶ MÉTRICAS DE TREINO:
  RMSE: 0.47

▶ MÉTRICAS DE VALIDAÇÃO:
  RMSE: 0.52


 ■ COMPARAÇÃO DE MODELOS (ordenados por RMSE de validação)

modelo  val_rmse
   KNN  0.517744


 ▶ MELHOR MODELO: KNN
  RMSE Validação: 0.52


In [16]:
melhor_resultado = df_resultados.iloc[0]
print(f"\n\n ▶ MELHOR MODELO: {melhor_resultado['modelo']}")
print(f"  RMSE Validação: {melhor_resultado['val_rmse']:,.2f}")



 ▶ MELHOR MODELO: KNN
  RMSE Validação: 0.52


---
# 3. Salvar Submissão & Fazer Log do Modelo

### Método para **guardar log das configurações do modelo**

In [17]:
def salvar_submissao_log(df_sub, modelo_treinado, nome_modelo, metricas):
    pasta = 'submissoes'
    os.makedirs(pasta, exist_ok=True)

    # 1. Listar ficheiros e encontrar o maior ID existente
    ficheiros = os.listdir(pasta)
    ids_existentes = []

    for f in ficheiros:
        # Verifica se o arquivo segue o padrão 'submission_X.csv'
        if f.startswith('submission_') and f.endswith('.csv'):
            try:
                # Extrai apenas o número do nome do arquivo
                # Ex: 'submission_12.csv' -> '12'
                numero_str = f.replace('submission_', '').replace('.csv', '')
                ids_existentes.append(int(numero_str))
            except ValueError:
                continue # Salta ficheiros que não tenham número válido

    # Se a lista estiver vazia, começa do 1. Se não, pega o maior + 1
    if not ids_existentes:
        next_id = 1
    else:
        next_id = max(ids_existentes) + 1

    # 2. Definir nomes dos ficheiros
    filename_csv = f"{pasta}/submission_{next_id}.csv"
    filename_json = f"{pasta}/submission_{next_id}_params.json"

    # 3. Salvar CSV
    df_sub.to_csv(filename_csv, index=False)

    # 4. Extrair Hiperparâmetros
    try:
        params = modelo_treinado.get_params()
    except:
        params = {"info": "Não foi possível extrair params"}

    # 5. Metadados
    metadados = {
        "id": next_id,
        "modelo": nome_modelo,
        "performance_validacao": metricas,
        "hiperparametros": params
    }

    # 6. Salvar JSON
    with open(filename_json, 'w', encoding='utf-8') as f:
        json.dump(metadados, f, indent=4, default=str)

    print(f"\n✓ Submissão #{next_id} salva com sucesso!")
    print(f" ▱ {filename_csv}")

In [18]:
melhor_modelo_nome = melhor_resultado['modelo']

melhor_modelo = modelos_treinados[melhor_modelo_nome]['modelo']

usar_scaled = modelos_treinados[melhor_modelo_nome]['usar_scaled']


if usar_scaled:
    X_test_usar = X_test_scaled

else:
    X_test_usar = X_test_aligned

pred_log = melhor_modelo.predict(X_test_usar)

pred_reais = np.expm1(pred_log)

# Submissão
df_submissao = pd.DataFrame({

    'id': df_teste.index,

    'price': pred_reais

})

salvar_submissao_log(df_submissao, melhor_modelo, melhor_modelo_nome, melhor_resultado.to_dict())


✓ Submissão #11 salva com sucesso!
 ▱ submissoes/submission_11.csv
